In [7]:
import pandas as pd
import numpy as np
import itertools
import random

# schema: R(A,B,C)
# query: select agg(C) from R where T.A=2000 and T.B=30;

def create_adult(): 
    df = pd.read_csv('adult.data',header=None,na_values=" NaN")
    # 1st column: age
    A = df.iloc[:, 0].astype('int64')
    #print(A)
    # 13th column: hours-per-week 
    B = df.iloc[:, 4].astype('int64')
    #print(B)
    #14th column: salary
    C = df.iloc[:, 10].astype('int64')
    print(C)
    data = {'A':A,'B':B,'C':C}
    return pd.DataFrame(data)

#synthetic data

dom_A = list(range(2000, 2024 + 1))
dom_B = list(range(18, 40 + 1))
doms=[dom_A,dom_B]

def create_db(size): 
    A = random.choices(range(2000,2024),k=size)
    B = random.choices(range(18,41),k=size)
    C = np.random.normal(1000,200,size)
    data = {'A':A,'B':B,'C':C}
    return pd.DataFrame(data)

table=create_db(10000)

true_dom_A = list(table['A'].drop_duplicates().values)
true_dom_B = list(table['B'].drop_duplicates().values)

In [8]:
def run_query(table,cond_A,cond_B,agg_C):
    if '=' in cond_A: 
        table=table[table['A']==int(cond_A.split('=')[1])]
    if '=' in cond_B: 
        table=table[table['B']==int(cond_B.split('=')[1])]
    if '<' in cond_A:
        if '<=' in cond_A:
            table=table[table['A']<=int(cond_A.split('<=')[1])]
        else: 
            table=table[table['A']<int(cond_A.split('<')[1])]
    if '>' in cond_A:
        if '>=' in cond_A:
            table=table[table['A']>=int(cond_A.split('>=')[1])]
        else: 
            table=table[table['A']>int(cond_A.split('>')[1])]
    if '<' in cond_B:
        if '<=' in cond_B:
            table=table[table['B']<=int(cond_B.split('<=')[1])]
        else: 
            table=table[table['B']<int(cond_B.split('<')[1])]
    if '>' in cond_B:
        if '>=' in cond_B:
            table=table[table['B']>=int(cond_B.split('>=')[1])]
        else: 
            table=table[table['B']>int(cond_B.split('>')[1])]
    result = 0
    if agg_C=='sum':
        result=table['C'].sum()
    if agg_C=='avg':
        result=table['C'].avg()
    if agg_C=='count':
        result=table['C'].count()
    #print('result', result)
    return result

#run_query(table,'A>2017','B>=34','sum')

In [9]:
def twoD_space_all(x, y): 
    return [(a, b) for a in x for b in y]

def twoD_space(table, query_cols):
    return list(map(tuple, table[query_cols].drop_duplicates().to_numpy()))

def twoD_utility(space, query):
    scores = [l1_distance(p1,query) for p1 in space]
    # Normalize scores
    mins = min(scores)
    maxs = max(scores)
    return [(s - mins)/(maxs-mins) for s in scores]

def l1_distance(p1,p2):
    return sum(abs(a - b) for a, b in zip(p1, p2))

def compute_sensitivity(space, scores): 
    neighbor_utilities = []
    for i in range(0,len(space)): 
        for j in range(i+1,len(space)):
            if l1_distance(space[i],space[j])<=1:
                neighbor_utilities.append(abs(scores[i]-scores[j]))
    return max(neighbor_utilities)

def twoD_exponential_mechanism(query,epsilon,space_fn):
    # Create the space and calculate the score for each element in space
    if space_fn==twoD_space_all:
        space=twoD_space_all(doms[0],doms[1])
    if space_fn==twoD_space:
        space = twoD_space(table,['A','B'])
    scores = twoD_utility(space, query)
    sensitivity = compute_sensitivity(space, scores)
    #print('sensitivity', sensitivity)
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    # Choose an element from R based on the probabilities
    return space[np.random.choice(list(range(0,len(scores))), 1, p=probabilities)[0]]

def accuracy(query,query_p,agg): 
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    result_p = run_query(table,'A='+str(query_p[0]),'B='+str(query_p[1]),agg)
    return result_p - result


def test_per_query(query,agg,count,space):
    errs = []
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    for i in range(count): 
        query_p = twoD_exponential_mechanism(query, 1, space)
        #print('query_p: select %s(C) from R where T.A=%d and T.B=%d;'%(agg,query_p[0],query_p[1]))
        result_p = run_query(table,'A='+str(query_p[0]),'B='+str(query_p[1]),agg)
        error = abs(result_p-result)
        errs+=[error]
    return min(errs), sum(errs)/len(errs), max(errs)

def test_per_query_adult(query,agg,count,space):
    errs = []
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    for i in range(count): 
        query_p = twoD_exponential_mechanism(query, 1, space)
        print('query_p: select %s(C) from R where T.A=%d and T.B=%d;'%(agg,query_p[0],query_p[1]))
        result_p = run_query(table,'A='+str(query_p[0]),'B='+str(query_p[1]),agg)
        error = abs(result_p-result)
        errs+=[error]
    return min(errs), sum(errs)/len(errs), max(errs)

def avg(l):
    return sum(l)/len(l)

def test(q_count): 
    agg = 'sum'
    min_err_all,max_err_all,avg_err_all=[],[],[] 
    min_err_res,max_err_res,avg_err_res=[],[],[] 
    for i in range(q_count):
        query = (random.sample(dom_A,1)[0],random.sample(dom_B,1)[0])
        print('query:   select sum(C) from R where T.A=%d and T.B=%d;'%(query[0],query[1]))
        print('correct result: %f'%(run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)))
        min_err,avg_err,max_err=test_per_query(query,agg,50,twoD_space_all)
        min_err_all+=[min_err]
        avg_err_all+=[avg_err]
        max_err_all+=[max_err]
        min_err,avg_err,max_err=test_per_query(query,agg,10,twoD_space)
        min_err_res+=[min_err]
        avg_err_res+=[avg_err]
        max_err_res+=[max_err]
    print('error: twoD_space_all: min:%f avg: %f max: %f'%(avg(min_err_all),avg(avg_err_all),avg(max_err_all)))
    print('error: twoD_space_res: min:%f avg: %f max: %f'%(avg(min_err_res),avg(avg_err_res),avg(max_err_res)))

          
test(3)

query:   select sum(C) from R where T.A=2022 and T.B=34;
correct result: 15139.991876
query:   select sum(C) from R where T.A=2000 and T.B=22;
correct result: 18474.698444
query:   select sum(C) from R where T.A=2012 and T.B=38;
correct result: 20907.260729
error: twoD_space_all: min:84.065155 avg: 6897.795238 max: 17109.061567
error: twoD_space_res: min:505.825682 avg: 4160.961913 max: 9700.540535
